<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/XRD_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XRDのグラフを作成するWebUIです

## <font color= “blue”>**※実行には測定データ(.txt)とピークファイル(.csv)が必要です！**</font>

## データの準備
1.   XRDの解析ソフトで測定データを読み込みこんで下さい
2.   最適化→**FP法**→**対数正規分布**を選択して実行ボタンを押してください
3.   自動検索から、元素を指定して結晶相を特定してください
4.   **ピークリストを保存**でピーク表(.csv)を保存してください
5.   測定ファイル(.raw)をアスキー変換してテキストファイル(.txt)に変換してください

## 利用方法

1.   <font color= “blue”>**前提ファイルの読み込み**</font>を実行する
2.   ディレクトリー名(`output_dir`)を入力して実行。左のフォルダーにdataなどが作成されます(**左のフォルダーアイコン**をクリックすると、エクスプローラーが開きます。フォルダーが見当たらないときは、上の<font color= “red”>**更新ボタン**</font>を押して下さい)
3.   data/[ディレクトリー名]/に**測定データ**(txtファイル)と**ピーク表**(csvファイル)をアップロード。<font color= “blue”>**ファイルを確認**</font>を実行して測定データとピークでそれぞれ**ファイルを選択**します

4.   ピークリストを確認して、正常に読み込めているか確認。もし**ファイル内すべてのスペクトルを重ねたい**ときは`All_Spectra`にチェック

5.   **グラフ作成**で指数付けがされたファイルを作製。また、自動でpng, pdf, svg形式で`out_put`に保存されます
6.   さらに詳細な設定をしたければ、**グラフの詳細設定**で作成

※ スペクトルを重ねても、指数付けは選択されたファイルのみ行います


In [1]:
#@title ##前提ファイルの読み込み (**とりあえず押して！**)

!pip install ipywidgets==7.7.1

from IPython.display import display
import pandas as pd
import numpy as np
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe

def round_to_sig(x, sig=3):
  return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)



# 出力をクリア
from IPython.display import clear_output
clear_output()



In [4]:
#@title ディレクトリー名の決定 { display-mode: "form"}
#@markdown ##実行ボタンを押してディレクトリー名を決定


#@markdown ##ディレクトリー名を入力
dir_name = "sample" # @param {"type":"string"}

os.makedirs("data", exist_ok=True)
dir_name_ud = os.path.join("data", dir_name)
os.makedirs(dir_name_ud, exist_ok=True)

print(f"ディレクトリ名: {dir_name}")

ディレクトリ名: sample


In [51]:
#@title ファイルを確認 { display-mode: "form"}
#@markdown ##1. 実行ボタンを押してファイルを確認
#@markdown ##2. 解析するファイルを選択 (実行ボタンは押さない)
files = os.listdir(dir_name_ud)
txt_files = [f for f in files if os.path.splitext(f)[1] == '.TXT']
txt_files = np.sort(txt_files)
peak_files = [f for f in files if os.path.splitext(f)[1] == '.csv']
peak_files = np.sort(peak_files)


print(f"ファイル数: XRDスペクトル...{len(txt_files)}, ピーク...{len(peak_files)}")

# ドロップダウンリストを作成
this_txt_file = widgets.Dropdown(
    options=txt_files,
    value=txt_files[0],
    description='XRDスペクトルファイルを選択:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)
this_peak_file = widgets.Dropdown(
    options=peak_files,
    value=peak_files[0],
    description='ピークファイルを選択:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)


# 表示
display(this_txt_file)
display(this_peak_file)

ファイル数: XRDスペクトル...2, ピーク...2


Dropdown(description='XRDスペクトルファイルを選択:', layout=Layout(height='40px', width='50%'), options=('MnZnFe_PEG_2024-…

Dropdown(description='ピークファイルを選択:', layout=Layout(height='40px', width='50%'), options=('Peaks_MnZnFe_PEG_2024…

In [ ]:
#@title ###ピークの確認
All_Spectra = False # @param {"type":"boolean"}

def getbk(text):
  start_index = text.rfind("(")
  end_index = text.rfind(")")
  return [start_index+1, end_index]

def read_peaks(filename, bk=0):
  peaks_raw = pd.read_csv(filename, encoding='shift_jis')

  peaks = peaks_raw[ ['2θ(deg)', '高さ(cps)', '相対高さ', '結晶相名' ] ]
  peaks = peaks.rename(columns={'2θ(deg)':'2theta','高さ(cps)':'high', '相対高さ':'high_r','結晶相名':'name'})

  peaks['name'] = peaks['name'].str.replace('),','):')
  new_rows = []
  for _, row in peaks.iterrows():
    if ':' not in row['name']:
      new_row = row.copy()
      new_rows.append(new_row)
    else:
      materials = row['name'].split(':')
      for material in materials:
        new_row = row.copy()
        new_row['name'] = material
        new_rows.append(new_row)

  peaks = pd.DataFrame(new_rows)
  peaks = peaks.sort_values(by=['2theta']).reset_index(drop=True)
  raw_name = peaks['name'].str
  split_df = peaks['name'].str.split('(', expand=True, n=1)
  peaks['material'] = split_df[1].str.strip()
  # Check if all unique materials are 'Unknown'
  if (split_df[0].unique() == 'Unknown').all():
    peaks['hkl'] = '000' # Set 'hkl' to '000' if all materials are Unknown
  else:
    for i, name in enumerate(peaks['name']):
      if '(' not in name:
        peaks.loc[i, 'hkl'] = '000'
        peaks.loc[i, 'material'] = 'Unknown'
      else:
        this_hkl_range = getbk(name)
        peaks.loc[i, 'hkl'] = name[this_hkl_range[0]:this_hkl_range[1]]
        peaks.loc[i, 'material'] = name[:(this_hkl_range[0]-1)]
    # peaks['hkl'] = split_df[bk].str.replace('\)', '', regex=True)
    peaks['hkl'] = peaks['hkl'].str.replace(',','')
  peaks = peaks.drop(columns=['name'])
  return peaks

def read_xrd(path):
  try:
    xrd_raw = pd.read_csv(path, encoding='utf-8', sep="\t",
                          skiprows=5, names=("2theta", "Intensity"))
  except:
    xrd_raw = pd.read_csv(path, encoding='utf-8', sep="\t",
                          skiprows=14, names=("2theta", "Intensity"))

  return xrd_raw

if All_Spectra:
  peak_path = []
  xrd_path = []
  for i in range(len(peak_files)):
    peak_path.append(os.path.join(dir_name_ud, peak_files[i]))
  for i in range(len(txt_files)):
    xrd_path.append(os.path.join(dir_name_ud, txt_files[i]))
else:
  peak_path = os.path.join(dir_name_ud, this_peak_file.value)
  xrd_path = os.path.join(dir_name_ud, this_txt_file.value)
Peaks = read_peaks(os.path.join(dir_name_ud, this_peak_file.value), bk=0)

display(Peaks)
XRD = read_xrd(os.path.join(dir_name_ud, this_txt_file.value))
plt.figure(figsize=(10, 5))
plt.plot(XRD['2theta'], XRD['Intensity'], label=os.path.basename(this_txt_file.value).replace(".TXT",""))
plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.xlim(min(XRD['2theta']), max(XRD['2theta']))
# plt.xticks(np.linspace(10, 80, 8) )
plt.legend()
plt.show()
#

In [ ]:
from operator import is_
from copy import Error
#@title ###グラフ作成

def plot_xrd(xrd_path, peaks_path, peak=False, label="_", unknown=False,
             rotation=0, Mark="", fontsize=10, fontblack=True, text_position="vertical",
             xrd_color="", offrate=30):

  raw_color = ['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B5']
  if xrd_color == "":
    xrd_color = raw_color[0:len(xrd_path)]
  else:
    if not isinstance(xrd_color, list):
      xrd_color = [xrd_color]*len(xrd_path)
  if not isinstance(xrd_path, list):
    xrd_path = [xrd_path]
  for i, this_path in enumerate(xrd_path):
    XRD = read_xrd(this_path)
    if label=="_":
      label=os.path.basename(this_path).replace(".TXT","")
    offset = i * max(XRD['Intensity']) * offrate / 100
    plt.plot(XRD['2theta'], XRD['Intensity'] + offset, label=label,
             color = xrd_color[i])
  if peak:
    if not isinstance(peaks_path, list):
      peaks_path = [peaks_path]
    Peaks = read_peaks(peaks_path[0])
    if isinstance(xrd_path, list):
      for i, this_path in enumerate(xrd_path):
        XRD = read_xrd(xrd_path[0])

    uni_mat = Peaks['material'].unique()
    Color_list = pd.DataFrame({'material': Peaks['material'].unique()})
    raw_color = plt.rcParams['axes.prop_cycle'].by_key()['color']
    unknown_rows = Color_list[Color_list['material'] == 'Unknown']
    known_rows = Color_list[Color_list['material'] != 'Unknown']
    Color_list = pd.concat([known_rows, unknown_rows], ignore_index=True)
    Color_list["color"] = raw_color[0:len(uni_mat)]
    if not unknown:
      Color_list = Color_list[Color_list['material'] != 'Unknown'].reset_index(drop=True)
    if fontblack:
      Color_list['color'] = "#000000"

    for mat in Color_list['material']:
      this_color = Color_list[Color_list['material'] == mat]['color'].values[0]
      dummy_handle, = plt.plot([], [], marker='*', linestyle='None',
                               color=this_color, label=f'index of {mat}')

    if not unknown:
      Peaks = Peaks[Peaks['material'] != 'Unknown'].reset_index(drop=True)

    font_size = fontsize
    old_x = 0
    count_y = 1
    for i, hkl in enumerate(Peaks['hkl']):
      this_text = f'({hkl})'
      if Mark != "":
        this_text = Mark
        font_size = fontsize * 1.5
      this_x = Peaks.loc[i,'2theta']
      idx = np.abs(XRD['2theta'] - this_x).idxmin()
      this_y = XRD.loc[idx, 'Intensity'] + (np.sin(np.deg2rad(rotation))/50 + 0.05) * max(XRD['Intensity'])
      this_y += (len(xrd_path) - 1) * max(XRD['Intensity']) / 20

      if this_x == old_x:
        if text_position == "vertical":
          this_y += font_size * count_y * (0.22 + np.sin(np.deg2rad(rotation)/3.2)) * max(XRD['Intensity']) / 50
          count_y += 1
        if text_position == "horizonal":
          this_x += font_size * (0.15 + np.cos(np.deg2rad(rotation))/5)
        else:
          Error("text_positionに誤りがあります")
      else:
        count_y = 1

      old_x = this_x
      plt.text(this_x, this_y, this_text, fontsize=font_size,
               horizontalalignment='center', verticalalignment='center',
               rotation=rotation, path_effects=[pe.withStroke(linewidth=3, foreground="white")],
               color=Color_list[Color_list['material'] == Peaks.loc[i,'material']]['color'].values[0])

#@markdown ### グラフの設定
#@markdown ピークを表示する
is_Peak = True # @param {"type":"boolean"}
#@markdown ピークの表示設定
Mark = "(hkl)" # @param ["(hkl)", "*", "o", "x"] {allow-input: true}
if Mark == "(hkl)": Mark = ""

plt.figure(figsize=(10, 5))
plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.yticks([])
plt.xlim(min(XRD['2theta']), max(XRD['2theta']))
plt.ylim(top=max(XRD['Intensity'])*(1.15 + np.sin(np.deg2rad(0)/6)))

plot_xrd(xrd_path, peak_path, peak=is_Peak, rotation=0, fontblack=False, text_position="vertical",
         Mark=Mark)
plt.legend()

if All_Spectra:
  save_file_name = "All_Spectra"
else:
  save_file_name = os.path.basename(this_txt_file.value).replace(".TXT","")

os.makedirs("output", exist_ok=True)
os.makedirs(os.path.join("output", dir_name) , exist_ok=True)

plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.png"))
plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.svg"))
plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.pdf"))

plt.show()


In [ ]:
from re import X
#@title ###グラフの詳細設定

#@markdown ###全般の設定
#@markdown　ピークを表示する
is_Peak = True # @param {"type":"boolean"}
#@markdown　ピークの表示設定
Mark = "(hkl)" # @param ["(hkl)", "*", "o", "x"] {allow-input: true}
if Mark == "(hkl)": Mark = ""
#@markdown　未知(Unknown)なピークの表示する
is_Unknow = False # @param {"type":"boolean"}

#@markdown ###XRDスペクトルの設定
#@markdown スペクトルの色 (入力しないと自動で色を設定)
XRD_Color = "" # @param {type:"string"}
#@markdown オフセット率
Offset = 30 # @param {type:"slider", min:0, max:100, step:1}

#@markdown ###ピークの設定
#@markdown フォントサイズ
Fontsize = 10 # @param {type:"slider", min:5, max:20, step:1}
#@markdown ミラー指数が結晶相で重なった際に並べる方向
Text_Position = "vertical" # @param ["vertical", "horizonal"] {allow-input: false}
#@markdown テキストの回転
Rotation = 45 # @param {type:"slider", min:0, max:360, step:1}
#@markdown テキストの色を全て黒に
is_fontblack = True # @param {"type":"boolean"}

#@markdown ###グラフ周辺の設定
#@markdown ラベル名 ("_"でファイル名がラベルになります)
Label = "This is Label" # @param {type:"string"}
#@markdown X軸のラベル名
X_Label = "2theta (deg)" # @param {type:"string"}
#@markdown Y軸のラベル名
Y_Label = "Intensity" # @param {type:"string"}
#@markdown 軸のフォントサイズ
Axis_Font_Size = 14 # @param {type:"slider", min:10, max:20, step:1}
#@markdown 凡例のフォントサイズ
Legend_Font_Size = 14 # @param {type:"slider", min:10, max:20, step:1}


plt.figure(figsize=(10, 5))
plt.xlabel(X_Label, fontsize=Axis_Font_Size)
plt.ylabel(Y_Label, fontsize=Axis_Font_Size)
plt.xlim(min(XRD['2theta']), max(XRD['2theta']))
plt.yticks([])

if Text_Position == "vertical":
  plt.ylim(top=max(XRD['Intensity'])*(1.15 + np.sin(np.deg2rad(Rotation)/6)))
else:
  plt.ylim(top=max(XRD['Intensity']) * 1.15)

plot_xrd(xrd_path, peak_path, peak=True, unknown=is_Unknow,
         rotation=Rotation, label=Label, fontblack=is_fontblack,
         text_position=Text_Position, fontsize=Fontsize,
         xrd_color=XRD_Color, offrate=Offset)

plt.legend(fontsize=Legend_Font_Size)

if All_Spectra:
  save_file_name = "All_Spectra"
else:
  save_file_name = os.path.basename(this_txt_file.value).replace(".TXT","")

os.makedirs("output", exist_ok=True)
os.makedirs(os.path.join("output", dir_name) , exist_ok=True)

plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.png"))
plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.svg"))
plt.savefig(os.path.join("output", dir_name, f"{save_file_name}.pdf"))

plt.show()